In [1]:
# Dependencies
import numpy as np
import pandas as pd
import sqlite3

In [2]:
# Create/Connect to Database
db = sqlite3.connect('CapstoneDatabase.sqlite')

In [3]:
# Instantiate Cursor
c = db.cursor()

In [4]:
# Load lifestyle data csv into Pandas dataframe
lifestyle_data = pd.read_csv("CleanedData.csv", index_col = 0)
lifestyle_data.shape

(1000, 14)

In [5]:
# Convert dataframe into a list of tuples
lifestyle_tuple = tuple(lifestyle_data.itertuples(index=False, name = None))

In [6]:
# Creating lifestyle table
c.execute("""CREATE TABLE lifestyle (
    patient_id text,
    age integer,
    gender integer,
    air_pollution integer,
    alcohol_use integer,
    dust_allergy integer,
    occupational_hazards integer,
    genetic_risk integer,
    chronic_lung_disease integer,
    balanced_diet integer,
    obesity integer,
    smoking integer,
    passive_smoking integer,
    cancer_severity integer
    )""")

OperationalError: table lifestyle already exists

In [ ]:
# Add data to lifestyle table
c.executemany("INSERT INTO lifestyle VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)", lifestyle_tuple)

In [7]:
# Load symptom data csv into Pandas dataframe
symptom_data = pd.read_csv("CleanedDataSymptoms.csv", index_col = 0)
symptom_data = symptom_data.drop(symptom_data.columns[3:13], axis=1)
symptom_data.shape

(1000, 15)

In [8]:
# Convert dataframe into a list of tuples
symptom_tuple = tuple(symptom_data.itertuples(index=False, name = None))

In [9]:
# Creating symptom table
c.execute("""CREATE TABLE symptoms (
    patient_id text,
    age integer,
    gender integer,
    chest_pain integer,
    coughing_of_blood integer,
    fatigue integer,
    weight_loss integer,
    shortness_of_breath integer,
    wheezing integer,
    swallowing_difficulty integer,
    finger_nail_clubbing integer,
    frequent_cold integer,
    dry_cough integer,
    snoring integer,
    cancer_severity integer
)""")

In [10]:
# Add data to symptoms table
c.executemany("INSERT INTO symptoms VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", symptom_tuple)

In [11]:
# Commit changes to the database
db.commit()

In [12]:
# Close database connection
db.close()